# Transformeer data

Combineer de data van de Kiesraad met die van Waar Is Mijn Stemlokaal en andere bestanden met postcodes, zodat we latitude en longitude kunnen vinden om te kunnen plotten op een kaartje. Daarna versimplificeren we de data en halen we er alleen de partijen uit die we echt nodig hebben.

In [52]:
# Data paths
DATA_PATH = "../1-data/ps2019-kiesraad.csv"
WIMS_PATH = "../1-data/wims.csv"
ZIP_PATH = "../1-data/dutch-pc6-zipcodes-with-lat-lon.csv"
OUT_PATH = "../3-pipeline/ps2019-kiesraad-with-locations.csv"
OUT_PATH_TMP = OUT_PATH.replace("pipeline/", "pipeline/tmp/")

Deze lijst bevat alle partijen die we meenemen in de analyse, op volgorde van het totaal aantal zetels wat ze landelijk gehaald hebben. Tevens zetten we hier de benaming uit de oorspronkelijke Kiesraad-data om naar kortere afkortingen. Er zijn twee lijstcombinaties die we samenvoegen: in Noord-Holland is er 50PLUS/Partij voor de Ouderen, die we optellen bij 50PLUS. ChristenUnie-SGP tellen we op bij zowel CU als de SPG.

In [53]:
PARTIES = {
 'FvD': 'Forum voor Democratie',
 'VVD': 'VVD',
 'CDA': 'CDA',
 'GL': 'GROENLINKS',
 'PvdA': 'Partij van de Arbeid (P.v.d.A.)',
 'D66': 'Democraten 66 (D66)',
 'PVV': 'PVV (Partij voor de Vrijheid)',
 'SP': 'SP (Socialistische Partij)',
 'CU': ['ChristenUnie', 'ChristenUnie-SGP'],
 'PvdD': 'Partij voor de Dieren',
 '50PLUS': ['50PLUS', '50PLUS/PARTIJ VAN DE OUDEREN'],
 'SGP': ['Staatkundig Gereformeerde Partij (SGP)', 'ChristenUnie-SGP'],
 'DENK': 'DENK'
}

Nu kunnen we echt gaan doen. Eerst importeren we alle noodzakelijke modules, daarna schonen we de postcodes op en voegen we de WaarIsMijnStemlokaal data toe.

In [54]:
from dataknead import Knead
import pandas as pd

In [55]:
def cleanup(row):
    # Remove spaces from zipcodes
    row["bureau_zip"] = row["bureau_zip"].replace(" ", "")
    return row

votes = Knead(DATA_PATH).map(cleanup)

In [56]:
# Add WIMS data
def parse(row):
    # Add Stembureau ID
    gemcode = row["CBS gemeentecode"].replace("GM", "")
    bcode = row["Nummer stembureau"]
    row["bureau_id"] = f"{gemcode}::SB{bcode}"
    return row

wims = Knead(WIMS_PATH).map(parse)

In [57]:
# Time to combine stuff, first convert wims to a dictionary because it's a lot faster for lookups
wims = {i["bureau_id"]:i for i in wims.data()}

def combine(row):
    bid = row["bureau_id"]

    if bid in wims:
        item = wims[bid]
        
        # We found something, always copy lat/lon, even if there's nothing
        row["bureau_lat"] = item["Latitude"]
        row["bureau_lon"] = item["Longitude"]

        # We prefer zips from the original data, but if there's none,
        # use WIMS
        if row["bureau_zip"] == "":
            row["bureau_zip"] = item["Postcode"]
                
    return row

votes = votes.map(combine)

In [58]:
# Now, for anything that's left, add the lat/lon from the zipdata that we have

# First, convert the zips to a dict again
zips = {z["zip"]:z for z in Knead(ZIP_PATH).data()}

# And now lookup the zips and save to a new file
def add_zip(row):
    if row.get("bureau_lat", "") == "":
        bzip = row["bureau_zip"]
        
        if bzip in zips:
            row["bureau_lat"] = zips[bzip]["lat"]
            row["bureau_lon"] = zips[bzip]["lon"]
            
    return row

votes = votes.map(add_zip)

# Save this file for debugging in the temp folder
votes.write(OUT_PATH_TMP)

Laad de data en transformeer naar een lijst met zeven waardes: 

* `label`: het label van het stembureau
* `party`: de partij waar op gestemd is
* `total_votes`: het totale aantal stemmen op het stembureau
* `votes`: het absolute aantal stemmen op de partij.
* `bureau_zip`, `bureau_lat` en `bureau_lon` om de data op een kaart te plotten

In [59]:
# Define an iterator to make things easier
def iter_votes_from_row(row):
    for partyId, partyLabels in PARTIES.items():
        # We can have multiple partyLabels, if we just have one, make 
        # a list of it for easier handling later on
        if isinstance(partyLabels, str):
            partyLabels = [ partyLabels ]
            
        for label in partyLabels:
            votecount = row[label]
            
            # Votecount is always a string. We need to be careful here, because
            # "0" is zero votes, while "" (an empty string) means that the 
            # party was *not* electable in this stembureau. Make sure we only 
            # yield actual votes!
            if votecount != "":               
                yield partyId, int(votecount)
        
# Now create the dataframe
elections = []

for row in votes.data():
    bureau, gemeente = (row["bureau_label"], row["gemeente"])
    label = f"{bureau.replace('Stembureau', '')} ({gemeente})"
    
    for party, votes in iter_votes_from_row(row):
        elections.append({
            "party" : party,
            "label" : label,
            "votes" : votes,
            "total_votes" : int(row["total_counted"]),
            "bureau_zip" : row["bureau_zip"],
            "bureau_lat" : row.get("bureau_lat", None),
            "bureau_lon" : row.get("bureau_lon", None)
        })
        
df = pd.DataFrame(elections)

# Save to a new file
df.to_csv(OUT_PATH)

# And show the first five results
df.head()

,bureau_lat,bureau_lon,bureau_zip,label,party,total_votes,votes
0,53.055922196,6.203015847,9244CR,"Beetsterzwaag, Gemeentehuis (Opsterland)",FvD,909,76
1,53.055922196,6.203015847,9244CR,"Beetsterzwaag, Gemeentehuis (Opsterland)",VVD,909,131
2,53.055922196,6.203015847,9244CR,"Beetsterzwaag, Gemeentehuis (Opsterland)",CDA,909,109
3,53.055922196,6.203015847,9244CR,"Beetsterzwaag, Gemeentehuis (Opsterland)",GL,909,117
4,53.055922196,6.203015847,9244CR,"Beetsterzwaag, Gemeentehuis (Opsterland)",PvdA,909,170
